<a href="https://colab.research.google.com/github/Charlee0616/Data-Mining/blob/main/Q%26Cproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split
from statistics import mean, stdev

In [34]:
def Attribute_selection_method(dataset, target, method, classification):
  if(method == "classification"): #method chosen was classification
    features = dataset.columns[dataset.columns != target]
    entropys=[]
    for attribute in features:
      entropy = 0
      overall = len( dataset )
      vals = dataset[attribute].unique()
      if(classification =="entropy"): #classifcation was chosen and entropy was chosen
        for val in vals:
          if(len(dataset[attribute].unique())<20):
            subset_size = len(dataset[ dataset[attribute] == val ])
            weight = subset_size / overall
            props = dataset[ dataset[attribute] == val ][target].value_counts( normalize=True )
            for p in props.array:
              entropy =  entropy - weight*(p*math.log2(p))
        else: #classification was chosen and Gini was chosen
          entropy = 0
          left = dataset[ dataset[attribute] <= val ][ [attribute,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          right = dataset[ dataset[attribute] > val ][ [attribute,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          entropys.append(entropy)
      else:
        entropy = 1 - sum([p ** 2 for p in props])
        entropy += weight * entropy
        entropys.append(entropy)
    minEnt=min(entropys)
    ind=entropys.index(minEnt)
    return features[ind]

  if(method == "NaiveBayes"): # method is Naive Bayes Classifier/ regressor
    #separate by class
    test = dataset.drop(columns=[target])
    separated = {}
    for i in range(len(dataset)):
      vector = dataset.iloc[i].values
      class_value = vector[-1]
      if (class_value not in separated):
        separated[class_value] = []
      separated[class_value].append(vector)

    #Summarize statistic
    summaries = {}
    for class_value, instances in separated.items():
      numerical_instances = [[float(x) for x in instance if isinstance(x, (int, float))] for instance in instances]
      summaries[class_value] = [(float(mean(col)), float(stdev(col)), len(col)) for col in zip(*instances)]


    predictions = []
    #put test into the method
    for index in range(len(dataset)):
        row = dataset.iloc[index, dataset.columns != target].values.astype(float) #access row data and convert to float
        total_rows = sum([summaries[label][0][2] for label in summaries])
        probabilities = {}
        for class_value, class_summaries in summaries.items():
          probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
          for i in range(len(class_summaries)):
            mean_val, stdev_val, _ = class_summaries[i]
            exponent = math.exp(-((row[i]-mean_val)**2 / (2 * stdev_val**2 )))
            probabilities[class_value] *=(1 / (math.sqrt(2 * math.pi) * stdev_val)) * exponent
        #determine the best label
        best_label, best_prob = None, -1
        for class_value, probability in probabilities.items():
          if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value

        predictions.append(best_label)
    return(predictions)

In [5]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv") # categorical dataset
StudentPerform = pd.read_csv("https://github.com/Charlee0616/Data-Mining/raw/main/StudentPerformanceFactors.csv") # numerical dataset
shrooms.describe()
StudentPerform.describe()


,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score
count,6607.000000,6607.000000,6607.00000,6607.000000,6607.000000,6607.000000,6607.000000
mean,19.975329,79.977448,7.02906,75.070531,1.493719,2.967610,67.235659
std,5.990594,11.547475,1.46812,14.399784,1.230570,1.031231,3.890456
min,1.000000,60.000000,4.00000,50.000000,0.000000,0.000000,55.000000
25%,16.000000,70.000000,6.00000,63.000000,1.000000,2.000000,65.000000
50%,20.000000,80.000000,7.00000,75.000000,1.000000,3.000000,67.000000
75%,24.000000,90.000000,8.00000,88.000000,2.000000,4.000000,69.000000
max,44.000000,100.000000,10.00000,100.000000,8.000000,6.000000,101.000000


In [18]:
CatentropyTest = Attribute_selection_method(shrooms, "class", "classification", "entropy")
CatentropyTest

'cap-shape'

In [16]:
NumentropyTest = Attribute_selection_method(StudentPerform, "Exam_Score", "classification", "entropy")
NumentropyTest

'Teacher_Quality'

In [35]:
CatRegressorTest = Attribute_selection_method(shrooms, "class", "NaiveBayes", "regression")
CatRegressorTest

IndexError: index 8 is out of bounds for axis 0 with size 8